In [19]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np

model_root = Path('./models')
model_name = 'GRU4REC_TOP1_Adagrad_0.05_epoch1'
model_file = model_root/model_name

dataset_root = Path('../sess-rec-large')
train = 'rsc15_train_full.txt'
test = 'rsc15_test.txt'
PATH_TO_TRAIN = dataset_root / train
PATH_TO_TEST = dataset_root / test

session_key = 'SessionId'
time_key = 'Time'
item_key = 'ItemId'

df_train = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
df_test = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [20]:
from modules.layer import GRU
from modules.model import GRU4REC
import torch

input_size = df_train[item_key].nunique()
output_size = input_size
hidden_size = 100
num_layers = 1
use_cuda = True
time_sort = False

dropout_hidden = .5
dropout_input = 0
batch_size = 50

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = 0, dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = 'TOP1', optimizer_type = 'Adagrad', lr=.05,
                time_sort = time_sort,
                pretrained = gru)

k = 20
recall, mrr = model.test(df_train, df_test, session_key,
                         time_key, item_key,
                         k = k, batch_size = 50)
result = f'Recall@{k}:{recall:.7f},MRR@{k}:{mrr:.7f}'
print(result)

Switching into Testing mode.
Switching into Training mode.
Recall@20:0.4141547,MRR@20:0.0357396


In [ ]:
# from pathlib import Path

# def save(model, PATH_TO_MODELS: str,  epoch: int):
#     '''
#     Args:
#         PATH_TO_MODELS: directory to store the models and the results
#         model: PyTorch model to save the parameters from
#         model_name (str): name of the model
#         epoch: index of the epoch
#     '''


#     if not PATH_TO_MODELS.exists():
#         PATH_TO_MODELS.mkdir()

#     subdirs = list(PATH_TO_MODELS.iterdir())
#     next_dir = str(int(max(subdirs).stem)+1) if len(subdirs)>0 else str(0)
#     next_dir = PATH_TO_MODELS/next_dir

#     PATH_ARGS = next_dir/'arguments.txt'
#     with open(PATH_ARGS, 'w') as f:
#         s = f'input_size:{input_size}\n'
#         s += f'output_size:{output_size}\n'
#         s += f'hidden_size:{hidden_size}\n'
#         s += f'num_layers:{num_layers}\n'
#         s += f'use_cuda:{use_cuda}\n'
#         s += f'time_sort:{time_sort}\n'
#         s += f'dropout_hidden:{dropout_hidden}\n'
#         s += f'dropout_input:{dropout_input}\n'
#         s += f'batch_size:{batch_size}'
#         f.write(s)

#     PATH_MODEL = next_dir/f'GRU4REC_epoch{epoch:d}'